In [1]:
from appgeopy import *
from my_packages import *

In [3]:
mainfld = r".\GWR_Prediction_Output"

subflds = glob(os.path.join(mainfld, "Layer*"))
subflds

['.\\GWR_Prediction_Output\\Layer_1',
 '.\\GWR_Prediction_Output\\Layer_2',
 '.\\GWR_Prediction_Output\\Layer_3',
 '.\\GWR_Prediction_Output\\Layer_4']

In [6]:
# select_fld = subflds[0]
for select_fld in subflds[:]:

    base = os.path.basename(select_fld)

    files_byFolder = sorted(
        glob(os.path.join(select_fld, "GWR_prediction*results.csv"))
    )

    output_combine = pd.concat(
        [pd.read_csv(f) for f in tqdm(files_byFolder)], ignore_index=True
    )

    output_combine = output_combine.iloc[:, :-2]
    output_combine["prediction_std"] = np.sqrt(output_combine["prediction_var"])

    output_combine = output_combine.sort_values(by="monthly").reset_index(
        drop=True
    )

    pointkey_arr = [
        f"X{int(x*1000)}Y{y*1000}"
        for x, y in zip(output_combine["X_TWD97"], output_combine["Y_TWD97"])
    ]
    output_combine.insert(loc=0, column="PointKey", value=pointkey_arr)

    output_combine.to_feather(os.path.join(mainfld, f"GWR_{base}.feather"))

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

In [11]:
select_pointkey = pointkey_arr[:5]

output_combine.query("PointKey in @select_pointkey & monthly<=4").to_csv("test_convert2netcdf.csv", index=False)